# PBMC Cell Type Markers



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scanpy.api as sc
import scipy as sp
import itertools
import numpy as np
import scipy.stats as stats
from scipy.integrate import dblquad
import seaborn as sns
from statsmodels.stats.multitest import fdrcorrection
import imp
import time
import pickle as pkl

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import sys
sys.path.append('/wynton/group/ye/mincheol/Github/scrna-parameter-estimation/simplesc')
import scme
import utils

In [4]:
data_path = '/wynton/group/ye/mincheol/parameter_estimation/interferon_data/'

In [5]:
%matplotlib inline

### Read the dataset

In [6]:
adata = sc.read(data_path + 'interferon.raw.h5ad')
adata = adata[:, adata.var.index.map(lambda x: x[:2] != 'HB')]
adata = adata[adata.obs['stim'] == 'ctrl', :].copy()

### Load test results

In [7]:
with open(data_path + 'ct_marker_1d.pkl', 'rb') as f:
    ct_marker_result = pkl.load(f)

In [135]:
'CD3D' in adata.var.index

True

In [137]:
np.where(adata.var.index == 'CD3D')

(array([1153]),)

In [138]:
ct_marker_result[('CD4 T cells', '-CD4 T cells')]['de_diff'][1153]

-1.3545278310775757

In [139]:
for ct in adata.obs.cell.drop_duplicates():

    imp.reload(utils)
    df = utils.get_differential_genes(
        adata.var.index,
        ct_marker_result, 
        ct, 
        '-' + ct,
        which='mean', 
        direction='decrease',
        sig=0.1,
        num_genes=1000)
    print(ct)
    print(df.head(55))

CD14+ Monocytes
      fdr  effect_size     genes  rank
1561  0.0     4.804954      CCL7     1
1664  0.0     4.790018  SERPINB2     2
113   0.0     4.330105    S100A8     3
465   0.0     4.328548     CXCL1     4
111   0.0     4.247978    S100A9     5
112   0.0     4.077067   S100A12     6
483   0.0     3.866310      HPSE     7
1175  0.0     3.736869    CLEC4E     8
468   0.0     3.573656     CXCL3     9
1731  0.0     3.513031  C19orf59    10
469   0.0     3.402577     CXCL2    11
545   0.0     3.393222      VCAN    12
652   0.0     3.390270    PLA2G7    13
1407  0.0     3.367114     THBS1    14
774   0.0     3.330626    CLEC5A    15
464   0.0     3.197627       IL8    16
270   0.0     3.156109      IL1B    17
1832  0.0     3.125443       OSM    18
612   0.0     3.010809      IER3    19
1841  0.0     2.993984    LGALS2    20
1258  0.0     2.922876       SDS    21
216   0.0     2.865401    CYP1B1    22
167   0.0     2.862392      G0S2    23
565   0.0     2.858573      CD14    24
971   0.0

In [117]:
imp.reload(utils)
df = utils.get_differential_genes(
    adata.var.index,
    ct_marker_result, 
    ct, 
    '-' + ct,
    which='mean', 
    direction='decrease',
    sig=0.05,
    num_genes=1000)

In [119]:
df.head(50)

,fdr,effect_size,genes,rank
420,0.0,5.636558,LAMP3,1
863,0.0,4.528530,IDO1,2
946,0.0,3.767452,TXN,3
625,0.0,3.618823,HLA-DQA1,4
462,0.0,3.376946,IGJ,5
623,0.0,3.369206,HLA-DRB5,6
788,0.0,3.326463,RAB9A,7
629,0.0,3.277359,HLA-DPA1,8
486,0.0,3.228488,DAPP1,9
630,0.0,3.213754,HLA-DPB1,10
